# Using Jurassic-2 Ultra on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Ultra** (formerly Jumbo Instruct) using Amazon SageMaker.

Jurassic-2 Ultra is nothing short of a revolution! This cutting-edge large language model by AI21 Labs excels at diverse tasks such as question answering, summarization, long-form copy generation, and information extraction. What’s more, it supports a range of non-English languages including Spanish, French, German, Portuguese, Italian, and Dutch. Its unmatched quality stands testament to its power, making it the most outstanding Jurassic model ever. Brace yourself for a thrilling AI experience!


## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Prompt with instructions](#C.-Prompt-with-instructions)
   1. [Prompt with examples](#D.-Prompt-with-examples)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:416407187090:model-package/j2-ultra-v1-1-053"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

### <span style='color:Blue'> How to choose the best instance for my use case?</span>
<span style='color:#0057FF'> When you create your endpoint, you need to choose the instance type to run the model on. Choosing the right instance is mainly a matter of economics. Depending on your use case, you probably want the most cost-effective instance possible. In this notebook we use one of the supported instances.</span>

<span style='color:#0057FF'>Looking for the list of all supported instances? See</span> [here](https://docs.ai21.com/docs/choosing-the-right-instance-type-for-amazon-sagemaker-models#jurassic-2-ultra).

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
endpoint_name = "j2-ultra"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"    # Recommended instance
#     "ml.g5.48xlarge"   # Cheaper and faster - recommended for relatively short inputs/outputs
#     "ml.p4de.24xlarge" # Recommended for long inputs/outputs and faster performance
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

----------------------------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Ultra as a smart auto-completion algorithm: give it some text as input and it will generate relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [9]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be
That is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Prompt with instructions

**Why**? This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

**When?** Drafting, seeking for inspiration, or when the format and guidelines are "work in progress".

**How?** Just provide an instruction.

For this notebook, we will apply the model to summarize chat conversations. The conversations are taken from the [samsum dataset](https://huggingface.co/datasets/samsum). We will start with providing the model a simple instruction.

In [11]:
chat_to_summarize = """Michael: Hey John!
John: Oh! Hi Michael
Michael: Why you always start conversation with these beautiful  gestures like oh!, O my goodness ?
John: Because i know that you always target me to discuss about your new car, isn't it?
Michael: Haha! yes, today i'll keep talking on same topic :p
John: O my goodness! then please start because you will definitely ruin my weekend :(
Michael: O this is not like that, i have decided to purchase the car, i delayed it due to some payment issues which now have been resolved. Now i can buy a new car.
John: That's great! Finally, you are all set to purchase the car.
Michael: O yes! my friend.
John: Do you remember? i used to say that you didn't have money for your daily survival then how could you purchase this car? Now! i am very happy to hear that you sufficient money to purchase that car.
Michael: So, would you go with me to receive Mercedes Benz 2.0 from showroom? 
John: Yes! my brother, it's a big day for you and i'll definitely go with you to receive this luxury Mercedes Benz.
Michael: But, there is a condition that, i will drive Mercedes Benz to home.
John: Haha!  Sure my friend.
Michael: i decided to get this darling car, but there are certain priorities which need to be fulfilled first.
John: You'll get your darling soon
Michael: Wow! (y)
John: OK then, see you on Saturday, Goodbye!
Michael: Sure! Goodbye."""

instruction = f"""Summarize the following conversation.
{chat_to_summarize}

Summary:
"""

In [13]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=200,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

Michael tells John that he finally has enough money to buy a new car, and that John can go with him to the dealership. John is happy for Michael and says that he will go with him.


#### Be specific in your prompt
The model was asked to summarize the chat conversation in a general way. Depending on your requirements, you may require a a different style, such as specific number of bullets. You can ask the model to stick to it (in this case, 2 bullets):

In [16]:
bullet_instruction = f"""Below is a conversation:
{chat_to_summarize}
##

Summarize the above conversation in 2 bullets.

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=bullet_instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

* Michael tells John that he finally has enough money to buy a new car.
* John is happy for Michael and agrees to go with him to the dealership.


#### Adjust the parameters
A useful parameter is the temperature. **You can increase creativity by tweaking the temperature.** With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation.
*Note: in tasks such as summarization, you should use low temperature, somewhere between 0-0.2*.

In [17]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=200,
                                   temperature=0.2,
                                   numResults=3) # this will make the model generate 3 optional completions

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

Michael tells John that he finally has enough money to buy a new car, and John agrees to go with him to the dealership.
Michael tells John that he is going to purchase a new car, a Mercedes Benz, and John agrees to go with him to the showroom.
Michael tells John that he finally has enough money to buy a new car, and that John should go with him to the dealership. John is excited and agrees.


#### Explore several phrasing
If you are working with instruction prompting, small changes in the phrasing of the instruction may have a significant impact on the completion. The following is another way of instruction:

In [18]:
another_prompt =f"""You are a professional summarizer, an AI assistant that excels at summarizing chats.
You will be presented with a conversation between two users. Read it and summarize it.

{chat_to_summarize}
--
Summary:
"""

In [21]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=another_prompt,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

Michael has decided to purchase a new car, a Mercedes Benz. John is happy for Michael and will go with him to receive the car.


### D. Prompt with examples

**Why?** Examples are helpful in assisting the model to comprehend and generate responses that adhere to the intended format.

**When?** Examples are particularly useful when there are stringent format constraints, a well-defined objective, and an overall structure to be maintained.

**How?** To establish a pattern for the model to follow, present a few instances (“shots”) of input-output pairs in the prompt. This enables the model to mimic the pattern. Then, provide the input for a query example and allow the model to generate a suitable completion. This approach is commonly referred to as a "*few-shot prompt*".

According to the specifications for this dataset, the summaries should: _"(1) be rather short, (2) extract important pieces of information, (3) include names of interlocutors, (4) be written in the third person"_. Using these examples, we can guide our model accordingly.

#### Create a few-shot prompt

We will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant inputs (a conversation) and the output (a summary). They are separated by "##".

2. The query inputs. An unseen chat conversation for which we would like the model to output a summary. These should follow the same format of the inputs in the prefix.

First, we collect some example data for the prompt prefix:

In [22]:
EXAMPLES_DATA = [
    {"chat": """Ash: Thanks so much! I am feeling so much better. I'm gonna work on the next chapter as soon as I can. I'm hoping to actually work on it tonight, but I have to beta another story. Thanks again!!!
Mollie: Please do! I love all your stories but this one cuts the cake. :) I'm glad to hear you're feeling better :D and have fun Beta-ing! :P
Mollie: P.S. Can I ask dor a small-ish favour? Could you make Clara say (in that last chapter): "run you clever boy" or something like that? If not I get it :P
Ash: I'm not sure if I can in the last chapter, BUT I was thinking about it...I might be able to make another itty bitty cameo with he...and I might be able to have her say it... I gotta think about how to incorporate it! 
Mollie: There is an option to edit chapters - it's really weird but manageable. Wither way, having Clara back would be exciting :D
Ash: Yeah... I know how to do that. I just wasn't sure if it made sense for her to say "clever boy." But I'm gonna try to have her pop back in. Maybe in the last chapter. Want to make such a lovely reviewer happy :)
Ash: Maybe this will make up for it until we get there? 
Ash: <file_other>
Mollie: Hey!! I'm so sorry I didn't reply earlier - I had a couple of trips going on and literally no time to check the Internet!!! :( Now my Inbox is stuffed with emails that need replying -_-
Mollie: Anyhoo, can I say how honoured I am that I got to see a sneak peek of the chapter (do you do this to all of your reviewers?? ;) ) and how much it made me want to read more! The chapter's lovely; I can't wait to see everyone;s reactions to Ian's presence in New York. :D
Mollie: One technical issue though - how on earth are you going to explain Ian's incredible late workload? Sorry, i'm just being picky... :P
Ash: I'm glad you like the preview! No I only give certain people previews. If their comments have really touched me then I usually try to show my thanks by sending a preview. Chapter 27 picks up a couple months after Ian and Roses reconciliation. Ian did work in certain parts, like all day in Little Talks and several others. But you are right that he is behind, and I will make a comment to address that! Gotta keep ya all happy!!! ;) 
Mollie: Out of curiosity though, are you ever going to have Ian tell Rose about switching realities experience??
Mollie: You could always have that as additional chapter, or you could have a chapter showing everyone's reactions to Ian and Rose being toegther (that would always be worth reading! :P) Either way, you have fun writing :D
Ash: It's not my plan to have him reveal it. It was about the realization for Ian. About him seeing the truth that's always been there. I never felt that he needed to tell her about it. He just needed her.
Mollie: Awww, that last line *Swoon*
Mollie: But sure, it is your story :) You can do whatever you want :P
Mollie: What I keep wondering is what will happen to Ian once he reached 2025 in his own timeline, how will he react ect. Sorry for bugging you like this, it's all very interesting :P 
Ash: I hope I didn't come across rude!!! I wasn't at all trying to do that!!! I've had that question before. It's all a bit timey wimey and weird. It's not like Quantum Leap where one replaces the original and the "original" is somewhere.
Ash: It's more of he was given this gift of jumping forward, so see what his life is/what it could be. But he had to make the decision if he was going to follow through. Was he going to make it happen, or was he going to give up the one thing in his life that made it precious? Was he going to fight for Rose? Was he going to believe that she loved him and allow himself that love? Paradox-y yes. Does that help? 
Mollie: No, you didn't come off as rude - if anything, it made me think to myself: well done, you've just offended the author" :P
Mollie: As for your explanation... er yeah kinda? I get why you did it for the plot (which also makes the plot more interesting), I'm just imaging Ian's reactions when he finally reaches 2025 and and thinks about how he woul have jumped timelines every day... am I making sense? Think of it as me creating a fanfic in my head of an actual fanfic :P""", 
     "summary": "Ash is feeling better and he will work on the next chapter as soon as he can. He has to beta another story tonight. Mollie likes all of his stories."},
    {"chat": """William: Hello Hobbs! A new update on our whereabouts. Just moved over to Merida after a long drive from Cancun. Empty roads giving you a weird feeling. We had all possible difficulties finding our airbnb accommodation as its address was like Marida, Caucel, Avenida 68, Puerta Sisal, Calle 31, Avenida Caucel 68, Calle 3/2419 B. After much asking around, as we don't have a navigation system here, and consulting our paltry maps, we gave up and phoned our host who navigated us to the house, which turned out to be located in a semi-fenced residential area outside the city. Quite a pretty and comfy place for that and entirely for us alone.
William: <file_photo>
William: So we finally had a goodnight's sleep, did some sightseeing in the capital of Yucatan today and are looking forward to reconnoiter the area tomorrow.
William: <file_photo>
Hobbs: Hi there! Many thanks for the details. Caucel rings a bell with me. Is it to the west of the city itself? A fairly new development?
William: That's it. Brand new and still being developed. Following a wide and brand new avenida today, we ended up with this. <file_photo> And dense vegetation beyond!
Hobbs: Looks like Mexico to me. But they'll pick up the continuation once they needed it. Practically minded as they are. And how did you like the city itself?
William: We got stuck on the Plaza Mayor, this being Sunday. It was turned into a market square with stalls offering local food, craftsmanship products and household goods, surprising un-touristy. In front of Palacio del Gobierno, a supreme show of dancing in local costumes.
William: <file_photo>
Hobbs: Did you manage to get to the balcony of the Palacio on the first floor and get some photos from there?
William: But of course!
William: <file_photo>
William: Theresa tried to prevent me from trespassing but you know me. Even managed to get to the loo there!
Hobbs: Good boy! And how about the cathedral? Casa de Montejo? Is it still accessible or is it only a bank?
William: Both, and Theresa intends to visit the cathedral again to take some more photos. I didn't find it that impressive, apart from the west facade of course. A small part of Casa de Montejo on the ground floor is a museum now, interiors with period furniture plus a small modern art gallery, quite pleasant though over-airconditioned, but the rest is a bank, I think.
Hobbs: If I remember correctly they bought the building some time ago and financed its restoration. To have a good overview of the city, you should try a ride on an open double-decker bus.
William: Planned for a day to come. Tomorrow we want to go to Sisal and drive along the coast to Progresso.
Hobbs: You'd better stay in Sisal, enjoy the beach and absolutely try one of their fish restaurants. As far as I remember the road leading eastwards peters out after a few kms and you are nowhere.  Years back with my team we mapped the ruins of Xcopte temple east of the road but it took us a strenuous march and even worse kayaking. I don't think it's been made open to sightseeing. I'd give it a miss anyway, nothing spectacular.
William: Thanks for the tip! We are planning though to drive down and visit Uxmal. Is it worth it?
Hobbs: Absolutely! It is a must. Huge, well preserved, with a totally unique - oval - pyramid, el Piramide del Adivino. Plan one whole day for this site. It is exceptional. One of my favourites in this part of Yucatan.
William: We will! I'm developing a slight liking for Maya architecture :-)
Hobbs: I bet you are! Anyway please keep me posted about your travelling there.
William: Sure. Your tips are invaluable to us. You'll hear from us soon!""", 
     "summary": "William and Theresa have just moved to Merida. They had problems getting there. They like their accommodation and they enjoy their sightseeing in the capital of Yukatan. They are planning to go to Sisal and Uxmal."},
    {"chat": """Diana: Hi Chris! Karen told me that you know someone from AIESEC
Chris: Hello Diana! Good to hear from you!
Chris: Not only I know someone from AIESEC, but I'm working with them myself :) How can I help you? Are interested in one of the projects?
Diana: Great! Thanks for replying so quickly :)
Diana: I was browsing your website and looking for different opportunities. I am particularly interested in Asia and Northern America, my only problem is that I don't speak other language than English
Chris: Ok, it doesn't necessarily mean that you won't be eligible to participate in projects in other countries. Especially in Asia we have a lot of opportunities for people who would like to teach English
Diana: That's great news! Won't it be a problem that I'm not a teacher? I don't have any experience
Chris: It all depends on what you would like to do, are you interested in being a volunteer or in something paid?
Diana: I'm interested in travelling that's for sure, so I really want to go somewhere. If you have something that will allow me to get paid for my work and will be in line with my experience, that'd be absolutely wonderful.
Diana: I'm open for being a volunteer as well - the only problem's that I may not be able to afford to pay for the accommodation.
Chris: Most of the organisations we're cooperating with cover the cost of the accommodation of our volunteers. You usually pay for a plane ticket and food. Certain organisations offer pocket money
Diana: Yes, I saw one really interesting offer from Japan. They're looking for someone who could teach English in high school, although they listed some nationalities - didn't mention British, quite surprising in my opinion. Does it mean that I can't apply?
Chris: Could you please send me a link? We have a lot offers and it'd be easier for me to advice you if I know the offer
Diana: Sure <file_other>
Chris: Thanks!
Chris: Okay, so, I think in this case it shouldn’t be a problem. You may not be regarded as a preferred candidate, but I don’t think that you’re not eligible. Also, if you’re really interested, I’ll gladly contact the organisers and ask if they are interested in your profile.
Diana: That would be amazing! Thank you Chris!
Chris: If this one doesn’t work out, and please, be aware that we’re in touch with people all over the world so it may happen that you won’t get the job you applied for, are you interested in another position?
Diana: I think I’ll be happy to participate in any project in Japan (as a volunteer or not), unless it’s related to computers, IT, technology, etc. as I have absolutely no clue about that stuff
Chris: And what about North America? You mentioned that you may be interested in this as well
Diana: Yes, I think it’s pretty much the same as with Japan. I’m not too keen on being a teacher at a camp, but everything other than that would be great
Chris: Got it! I will look for something suitable. May you send me your CV?
Diana: <file_other>
Chris: Thanks! If you have any questions, do not hesitate to ask :)
Diana: Thank you! I really appreciate your help Chris. The whole idea is brilliant, but I admit it’s a bit overwhelming and confusing especially as you’re new to the whole thing
Chris: No worries, I've heard that a lot and we're doing everything we can to facilitate the process and make it more user friendly
Chris: We also highly recommend everyone to visit us in one of our offices, so if you live in London, I'll be happy to meet you
Diana: Perfect!""", 
     "summary": "Chris is working with the AIESEC. Diana is interested in AIESEC's projects in Asia and Northern America. She sent Chris an offer in which they are looking for an English teacher in high school in Japan. Chris will contact the organisers on behalf of Diana and will let her know what their answer is."},
]

Then, we use the following helper functions to construct the prefix:

In [23]:
def make_single_example(chat, summary):
    example = "You are a professional summarizer, an AI assistant that excels at summarizing chats.\nYou will be presented with a conversation between two users. Read it and summarize it.\n\n"
    example += chat
    example += "\n--\n"
    example += "Summary:\n"
    example += summary
    
    return example

SEPARATOR = "\n\n##\n\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["chat"], x["summary"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [24]:
def create_chat_summary_prompt(chat):
    """
    Create a few-shot prompt to generate a chat summary with Jurassic-2 models
    The prompt contains a preset sequence of examples followed by the query chat
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(chat, '')  # keep the summary blank and let the model generate

Let's see how this looks for the chat from before:

In [25]:
few_shot_prompt = create_chat_summary_prompt(chat=chat_to_summarize)

print(few_shot_prompt)

You are a professional summarizer, an AI assistant that excels at summarizing chats.
You will be presented with a conversation between two users. Read it and summarize it.

Ash: Thanks so much! I am feeling so much better. I'm gonna work on the next chapter as soon as I can. I'm hoping to actually work on it tonight, but I have to beta another story. Thanks again!!!
Mollie: Please do! I love all your stories but this one cuts the cake. :) I'm glad to hear you're feeling better :D and have fun Beta-ing! :P
Mollie: P.S. Can I ask dor a small-ish favour? Could you make Clara say (in that last chapter): "run you clever boy" or something like that? If not I get it :P
Ash: I'm not sure if I can in the last chapter, BUT I was thinking about it...I might be able to make another itty bitty cameo with he...and I might be able to have her say it... I gotta think about how to incorporate it! 
Mollie: There is an option to edit chapters - it's really weird but manageable. Wither way, having Clara b

In [26]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=few_shot_prompt,
                                   maxTokens=200,
                                   temperature=0.2,
                                   numResults=3) # this will make the model generate 3 optional completions

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

Michael and John are discussing about Michael's new car. Michael has decided to purchase the car and is waiting for Saturday to receive it. John is happy for Michael and will go with him to receive the car.
Michael is buying a new car, a Mercedes Benz. John is happy for Michael and will go with him to receive the car.
Michael and John are talking about Michael's new car. Michael has decided to purchase the car and is waiting for Saturday. John is happy for Michael and will go with him to receive the car.


As you can see, the completions follow a similar pattern to the examples in the few-shot prompt.

### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [27]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [28]:
model.delete_model()